# The purpose of this notebook is to explore visualization for event-triggered spiking within ntrode, across events

In [49]:
%install_ext https://raw.githubusercontent.com/SiggyF/notebooks/master/pep8_magic.py

UsageError: Line magic function `%install_ext` not found.


In [48]:
import loren_frank_data_processing as lfdp
import replay_classification as replay
import ripple_detection as ripple
import matplotlib.pyplot as plt
import holoviews as hv
import inspect
import os
import pandas as pd
import numpy as np
from ipywidgets import FloatProgress
# jupyter nbextension install --user pep8_magic.py
%load_ext pep8_magic
# from IPython.display import display
hv.notebook_extension('bokeh')

ModuleNotFoundError: No module named 'pep8_magic'

In [5]:
animal_id = 'JZ1'
day, epoch = [1, 2]
ripple_area = 'ca1'
mu_areas = ['ca1', 'mec']
animals = {
    'JZ1': lfdp.Animal(short_name='JZ1', directory='../Raw-Data/JZ1')}
date = 20161114
epoch_index = ('JZ1', 1, 2)
areas = ['ca1']
mark_variables = ['channel_1_max', 'channel_2_max', 'channel_3_max',
                  'channel_4_max']

## ntrode info

In [10]:
ntrode_df = lfdp.make_tetrode_dataframe(animals)
ntrode_keys = ntrode_df.xs(epoch_index, drop_level=False).index.values
ntrodes = ntrode_df.xs(epoch_index, drop_level=False).index.get_level_values('tetrode_number').tolist()
mec_ntrode_df = ntrode_df.xs(epoch_index, drop_level=False).query('area == "mec"')
ca1_ntrode_df = ntrode_df.xs(epoch_index, drop_level=False).query('area == "ca1"')

## Position

In [8]:
position_variables = ['linear_distance', 'trajectory_direction', 'speed']
position_df = lfdp.get_interpolated_position_dataframe(epoch_index, animals)
position_df['time'] = position_df.index #add the time index values as another col

## ripple_times

In [11]:
hippocampus_lfps = lfdp.get_LFPs(ca1_ntrode_df.index, animals)
time = hippocampus_lfps.index
sampling_frequency = 1500
ripple_times = ripple.Kay_ripple_detector(
        time, hippocampus_lfps.values, position_df.speed.values, sampling_frequency,
        minimum_duration=pd.Timedelta(milliseconds=15), zscore_threshold=4)

## MultiUnit Spikes

In [12]:
f = FloatProgress(min=0, max=ntrode_keys.shape[0])
f.value = 0
display(f)
mu_df = []
for key_num, ntrode_key in enumerate(ntrode_keys):
    mu_df.append(lfdp.get_multiunit_indicator_dataframe(ntrode_key, animals))
    f.value += 1

FloatProgress(value=0.0, max=30.0)

# Peri-swr spiking per ntrode

In [19]:
ca1_ntrode_df.index.values

array([('JZ1', 1, 2, 16), ('JZ1', 1, 2, 17), ('JZ1', 1, 2, 18),
       ('JZ1', 1, 2, 19), ('JZ1', 1, 2, 20), ('JZ1', 1, 2, 21),
       ('JZ1', 1, 2, 22), ('JZ1', 1, 2, 23), ('JZ1', 1, 2, 24),
       ('JZ1', 1, 2, 25), ('JZ1', 1, 2, 26), ('JZ1', 1, 2, 27),
       ('JZ1', 1, 2, 28), ('JZ1', 1, 2, 29), ('JZ1', 1, 2, 30)],
      dtype=object)

In [40]:
# f = FloatProgress(min=0, max=len(ca1_ntrode_keys))
# f.value = 0
# display(f)
ca1_ntrode_keys = ca1_ntrode_df.index.values
window_offset = [-0.5, 0.5]
sampling_frequency = 1500
from src.visualization import plot_perievent_raster
ca1_keys = ntrode_df.query('area == "ca1"').index
num_ntrodes = len(ca1_ntrode_keys)
col_wrap = 5
n_rows = np.ceil(num_ntrodes / col_wrap).astype(int)

In [26]:
len(axes.ravel())

15

In [32]:
len(ca1_ntrode_keys)

15

In [39]:
events

,start_time,end_time
ripple_number,,
1,00:28:07.081867,00:28:07.135200
2,00:28:09.424533,00:28:09.501200
3,00:28:13.159200,00:28:13.207200
4,00:28:14.932533,00:28:15.001200
5,00:28:16.880533,00:28:16.961867
6,00:28:19.627867,00:28:19.706533
7,00:28:34.423200,00:28:34.734533
8,00:28:39.040533,00:28:39.099867
9,00:28:39.481200,00:28:39.554533


# matplotlib

In [47]:
import src.visualization as viz
fig, axes = plt.subplots(n_rows, col_wrap, figsize=(col_wrap * 3, n_rows * 3), sharex=True, sharey=True)
ntrode_keys = ca1_ntrode_keys
for ntrode, ax, tetrode_key in zip(np.arange(len(ntrode_keys)), axes.ravel(), ntrode_keys):
#     plot_perievent_raster(ntrode_key, animals, ripple_times,
#                           ntrode_df, window_offset=(-0.5, 0.5),
#                           ax=ax, s=1)
    spikes = mu_df[ntrode]['channel_1_max'].notnull().astype('int')
    events = ripple_times
    event_locked_spikes = lfdp.reshape_to_segments(spikes, events,
                                                   window_offset=window_offset,
                                                   sampling_frequency=sampling_frequency).unstack(level=0).fillna(0)
    time = event_locked_spikes.index.total_seconds()
    spike_index, event_number = np.nonzero(event_locked_spikes.values)

    ax.scatter(time[spike_index], event_number)
    ax.axvline(0.0, color='black')
    ax.set_title(
        ntrode_df.loc[ca1_ntrode_keys[ntrode][:4]].area.upper() + ' - ' +
        str(ca1_ntrode_keys[ntrode]))
    ax.set_ylabel(events.index.name)
    ax.set_xlabel('time (seconds)')
    ax.set_ylim((0, events.index.max() + 1))
    ax.set_xlim(window_offset)

    ax2 = ax.twinx()
    kde = viz.kernel_density_estimate(event_locked_spikes, sampling_frequency, sigma=0.025)
    m = ax2.plot(time, kde[:, 1], color='blue', alpha=0.8)
    ax2.fill_between(time, kde[:, 0], kde[:, 2], color=m[0].get_color(), alpha=0.2)
    ax2.set_ylabel('Firing Rate (spikes / s)')

In [45]:
    
for ax in axes.ravel()[len(ntrode_keys):]:
    ax.axis('off')
plt.tight_layout()

In [ ]:
# import src.visualization as viz
# os.system('subl {0}'.format(viz.__file__))